In [35]:
import requests
from lxml import etree
import re
import json
import pandas as pd

In [36]:
rw_csv = pd.read_csv('rw_csv.csv')

In [37]:
falcon_url = "https://www.inoreader.com/stream/user/1004920918/tag/rw_monitor%3A%20FalconFeedsIO/view/json"

# fetch the RSS feed
response = requests.get(falcon_url)
rss_json = response.json()

In [38]:
rw_list = []
for item in rss_json['items']:
    if "#ransomware" in item['title']:
        tweet = item['title'].replace('\n',' ')
        link = item['url']
        ransomware = tweet.split('#ransomware')[0]
        date_published = item['date_published'][:10]
        date_published = pd.to_datetime(date_published).strftime('%Y-%m-%d')
        pattern = r"added (.*?) to their victim list"
        try:
            victim = re.search(pattern, tweet).group(1)
            if "," in victim:
                victim = victim.split(',')[0]
            rw_list.append({'victim': victim,
            'ransomware': ransomware,
            'date_published':date_published,
            'link':link})
        except:
            next
                # print(f'Ransomware: {ransomware}\nVictim: {victim}\nLink: {link}\nDate: {date_published}')
        
# rw_df = pd.DataFrame(rw_list)
# rw_df

In [39]:
threatmon_url = 'https://www.inoreader.com/stream/user/1004920918/tag/rw_monitor%3A%20threatmon/view/json'

# fetch the RSS feed
response = requests.get(threatmon_url)
rss_json = response.json()

In [40]:
# rw_list = []
for item in rss_json['items']:
    if "Actor :" in item['title']:
        tweet = item['title'].replace('\n',' ')
        link = item['url']
        ransomware = tweet.split(' Victim :')[0].replace("Actor : ",'').strip()
        date_published = pd.to_datetime(item['date_published'][:10]).strftime('%Y-%m-%d').strip()
        try:
            victim = tweet.split(' Victim : ')[1].split(" Date")[0].strip()
            # print(f'{ransomware} | {victim} | {date_published}')
            rw_list.append({'victim': victim,
            'ransomware': ransomware,
            'date_published':date_published,
            'link':link})
        except:
            next
rw_df = pd.DataFrame(rw_list)
rw_df

,victim,ransomware,date_published,link
0,Création Baumann AG (creationbaumann.com),BlackByte,2023-04-10,https://twitter.com/FalconFeedsio/status/16453...
1,City of Collegedale (collegedaletn.gov),BlackByte,2023-04-10,https://twitter.com/FalconFeedsio/status/16453...
2,Scantibodies Laboratory,Medusa,2023-04-10,https://twitter.com/FalconFeedsio/status/16452...
3,Grupo Cbb (cbb.cl),BlackByte,2023-04-10,https://twitter.com/FalconFeedsio/status/16452...
4,Crown Grinding & Machining (crowngrinding.com),BlackByte,2023-04-10,https://twitter.com/FalconFeedsio/status/16452...
5,Palo Alto Sheriff (paloaltosheriff.com),PLAY,2023-04-09,https://twitter.com/FalconFeedsio/status/16449...
6,PKF Antares (pkfantares.com),PLAY,2023-04-09,https://twitter.com/FalconFeedsio/status/16449...
7,Legion Aero (legionaero.com),PLAY,2023-04-09,https://twitter.com/FalconFeedsio/status/16449...
8,Michiels Vleeswaren,PLAY,2023-04-09,https://twitter.com/FalconFeedsio/status/16449...
9,Schirm (schirm.com),PLAY,2023-04-09,https://twitter.com/FalconFeedsio/status/16449...


In [41]:
df_combined = pd.concat([rw_df,rw_csv],ignore_index=True)
df_combined.drop_duplicates('link',inplace=True)
rw_df.to_csv('rw_csv.csv', mode='w', header=True, index=False)